In [1]:
from random import *
from Crypto.Util.number import getPrime, isPrime
from math import gcd
from math import log2
#from Crypto.Hash import SHA3_256
from hashlib import sha3_256
from math import floor
m="allo"
h = sha3_256(bytes(m, "utf-8")).hexdigest()

Informations courbes ellipitues

In [ ]:
#je met les paramétres de la courbe question 12 ici car ça me permet de tout executer

a1=57896044618658097711785492504343953926634992332820282019728791901641727051837
b1=398341948620716521344
q1 = 2**252 + 27742317777372353535851937790883648493
G1=(2, 2587177637973221124604506650587198648324617568965701997790174145338249409797)
p1=(2**255-19)


EC25519=[p1,a1,b1,G1,q1]

ECbis=[23,1,2,[2,14],12]

EC=[19,1,1,[16,16],21] 

EC_sous_groupe=[19,1,1,[14,2],7]

Fonction qui implémente l'algorithme d'Euclide étendu.  Elle calcule le PGCD de a et b, ainsi que les coefficients de Bézout x et y.

In [5]:
#fonction outil à ne pas prendre en compte ( créer dans l'ancien dm)
def xgcd(a,b): 
  x,y=1,0
  u,v=0,1
  while b != 0:
   q,r=a//b,a%b
   m,n=x-u*q,y-v*q #On fait l'opération L1-q*L2 (1)
   a,b=b,r #On fait un décalage car la technique le veut. Le quotient devant le numérateur et le reste devient le quotient 
   x,y=u,v #Maintenant comme on fait L2-L3 les coefficients ne sont plus les mêmes pour la ligne de calcul(1), 
   u,v=m,n #donc on va echanger.
  reste=a
  return [reste,x,y] #a est le reste b 

# Début TP

Concernant les points d’une courbe, nous en choisirons une représentation affine, à savoir un couple $(x,y) \in F_p^2$
satisfiant l’équation de la courbe. Le seul point qui ne rentrera pas dans le cadre de cette
représentation est le point à l’infini, que nous représenterons par la chaîne de caractères "ZERO". Ainsi,
nous devrons créer deux fonctions particulières :
1. Une fonction ec_zero() qui retourne la représentation de ce point à l’infini (le neutre du groupe de
points, autrement dit zéro), donc qui retourne la chaîne de caractères "ZERO" ;
2. Une fonction ec_is_zero(P) qui retourne True si l’objet P passé en paramètre est égal à notre
représentation du point à l’infini, et qui retourne False sinon.

Question 1.– Implanter les deux fonctions ec_zero() et ec_is_zero(P) décrites ci-dessus, puis vérifier
que ec_is_zero(ec_zero()) retourne bien True.

In [6]:
#Question 1

def ec_zero():
    return "ZERO"
    
def ec_is_zero(P):
    if P==ec_zero():
        return True
    else :
        return False
    
ec_is_zero(ec_zero())

True

### Question 2.– 
IImplanter une fonction ec_opp(EC, P), qui prend en entrée la donnée d’une courbe elliptique EC et un point P de la courbe, et qui retourne l’opposé du point P (pour l’opération de groupe). On
prendra garde à distinguer le cas où P est un point affine de la courbe, du cas où P est le point à l’infini.

In [7]:
#Question 2

def ec_opp(EC,P):
    p=EC[0]
    if ec_is_zero(P)==True:
        return ec_zero()
    elif P=="ZERO":
        return ec_zero()
    else:
        L=P.copy()
        L[1]=-L[1]%p #Question : operation de groupe, comment faire pour distinguer dans EC l'opération de groupe.
    return L

A =[1,10]

ec_opp(EC,A)




[1, 9]

In [ ]:

def inverse_modulaire(a,b):
    coeffBezout=xgcd(a,b)
    if coeffBezout[0]==1:
        return coeffBezout[1]
    else:
        print("a et b ne sont pas premier entre eux")

### Question 3.– 
Implanter une fonction ec_double(EC, P), qui prend en entrée la donnée d’une courbe
elliptique EC et un point P de la courbe, et qui retourne le double du point P (pour l’opération de groupe).
On prendra garde à distinguer le cas où P est égal à son opposé des autres cas.

In [10]:
#Question 3
def ec_double(EC, P):
    p = EC[0]
    if ec_is_zero(P) == True:
        return ec_zero()
    elif P[1] % p == -P[1] % p:
        return ec_zero()

    else:
        inv = inverse_modulaire(2 * P[1], p)
        u = ((3 * pow(P[0], 2, p) + EC[1]) * inv) % p
        x3 = (pow(u, 2, p) - 2 * P[0]) % p
        y3 = (-P[1] + u * (P[0] - x3)) % p
        return [x3, y3]

# Example usage:
EC = [19, 1, 1, [16, 16], 21]
P = [5, 6]
print(ec_double(EC, P))


[9, 13]


In [11]:
ec_double(ECbis,[3,3])

[3, 20]

### Question 4.– 
Implanter une fonction ec_add(EC, P, Q), qui prend en entrée la donnée d’une courbe
elliptique EC et deux points P et Q de la courbe, et qui retourne la somme des points P et Q (pour l’opération
de groupe). Comme pour les questions précédentes, certains cas seront à distinguer.

In [12]:
#Question 4
def ec_add(EC,P,Q) :
    p=EC[0]
    if ec_is_zero(P)==True and ec_is_zero(Q)==True:
        return ec_zero()
    elif ec_is_zero(P)==True and ec_is_zero(Q)==False:
        return Q
    elif ec_is_zero(P)==False and ec_is_zero(Q)==True:
        return P
    elif Q==ec_opp(EC,P) :
        return ec_zero()
    elif P==Q :
        return ec_double(EC,P)
    
    else:
        if xgcd(Q[0]-P[0],p)[0]!=1:
            print("Le reste n'est pas égale à 1 mais à :",xgcd(Q[0]-P[0],p)[0])
        u=(Q[1]-P[1])*inverse_modulaire(Q[0]-P[0],p)
        v=(P[1]-u*P[0])
        x3=(u**2-P[0]-Q[0])
        y3=(-P[1]+u*(P[0]-x3))
        
        return [x3%p,y3%p]
    

ec_add(ECbis,[2,14],[1,2])


[3, 20]

### Question 5.– 
Implanter une fonction ec_fast_mult(EC, P, m), qui prend en entrée la donnée d’une
courbe elliptique EC, d’un point P de la courbe et d’un entier m, et qui retourne le point mP, c’est-à-dire le
m-ème itéré du point P. Votre fonction devra utiliser la méthode double-and-add vue en cours, qui adapte
la méthode d’exponentiation rapide au cas d’un groupe additif.

In [13]:
#Question 5
def ec_fast_mult(EC, P, m):
    if P== ec_zero() :
        return P
    else :
        p=EC[0]
        nombre_binaire = str(bin(m)[2:])
        m_liste_binaire = [int(chiffre) for chiffre in nombre_binaire]
        liste_binaire=m_liste_binaire[::-1]
        R=ec_zero()
        Q=P.copy()
        #print("Q=",Q)
        L=[]
        for i in range(0,len(liste_binaire)):
            if liste_binaire[i]==1:
                R=ec_add(EC,R,Q)
            
            Q=ec_add(EC,Q,Q)
            L.append(Q)

        return R

### Question 6.– 
Créer une variable chaine_test contenant la chaîne de caractères

"Cryptographie à clé publique"
(sans les guillemets), et vérifier que son haché par la fonction SHA3-256 est, en écriture hexadécimale :
2175c8e979e092c9dc9010e1c06ea490dff52f1bf5d9df8fd2b7f35b2ae58917

In [ ]:
from hashlib import sha3_256

chaine_test=str('Cryptographie à clé publique')
h3 = sha3_256(bytes(chaine_test, "utf-8")).hexdigest()
print(h3=="2175c8e979e092c9dc9010e1c06ea490dff52f1bf5d9df8fd2b7f35b2ae58917")

True


### Question 7.– 

Convertir le haché obtenu dans la question précédente en un entier, et vérifier que cet entier
vaut
15134431753598964768903003036524780383794702900043419402360626898381641713943

In [46]:
#Question 7
h3_int=int(h3,16)
print(h3_int==15134431753598964768903003036524780383794702900043419402360626898381641713943)

True


### Question 8.– 
Soit q = 1000003. Hacher le contenu du fichier testfile.txt disponible à l’adresse suivante :
https://lvzl.fr/teaching/2023-24/docs/cp/testfile.txt
Puis, vérifier que le haché, converti en entier, puis réduit modulo q, vaut 943673.

In [47]:

OuvrirTestFile = open("testfile.txt", 'r')

text_testfile = OuvrirTestFile.read()


#Question 8
hache_text=sha3_256(bytes(text_testfile, "utf-8")).hexdigest()

hache_nombre=int(hache_text,16)%1000003
print(hache_nombre==943673)

True


### Question 9.– 
Implanter une fonction ecdsa_keygen(EC), qui prend en entrée la donnée d’une courbe
elliptique EC, et qui retourne une paire de clés publique/privée du système de signature ECDSA.

In [20]:
#Question 9
def ecdsa_keygen(EC) :
    G,q=EC[-2:]
    p=EC[0]
    a=randint(1,q-1) 

    A=ec_fast_mult(EC,G,a)

    return[A,a] #A:cle publique et a:cle secrete 




### Question 10.– 
Implanter une fonction ecdsa_sign(EC, message, sk), qui prend en entrée la donnée
d’une courbe elliptique EC, une chaine de caractères message et une clé privée sk, et qui retourne une
signature du message par la clé privée dans le système de signature ECDSA.

In [21]:
#Question 10

import random 

def ecdsa_sign(EC,m,sk):
    p,q,G=EC[0],EC[-1],EC[-2]
    h = sha3_256(bytes(m, "utf-8")).hexdigest()
    h1=int(h,16)%q
    b,c=0,0

    E=[]
    while b==0 or c==0 :
        k=randint(1,q-1) #On a suppose que q soit premier donc pas besoin de prendre un nombre inversible à
        B=ec_fast_mult(EC,G,k)
        
        print(B,k)
        b=B[0]%q
        c=((h1+sk*b)*inverse_modulaire(k,q))%q
    return [b,c]

In [48]:
pk,sk=ecdsa_keygen(EC_sous_groupe)


#il faut le mettre là parce que les fonctions sign ne sont pas deterministe(a cause de l'alea)
s=ecdsa_sign(EC_sous_groupe,chaine_test,sk)
print(s)


[10, 17] 2
[3, 6]


In [49]:
#verification des fonctions mult et add

G = EC_sous_groupe[-2]
print([ec_fast_mult(EC_sous_groupe, G, i) for i in range(21)])
Liste_cyclique=[(inverse_modulaire(k, 7)*k) % 7 for k in range(1,7)]

ec_add(EC_sous_groupe,"ZERO",[14,17])


['ZERO', [14, 2], [10, 17], [15, 16], [15, 3], [10, 2], [14, 17], 'ZERO', [14, 2], [10, 17], [15, 16], [15, 3], [10, 2], [14, 17], 'ZERO', [14, 2], [10, 17], [15, 16], [15, 3], [10, 2], [14, 17]]


[14, 17]

### Question 11.– 
Implanter une fonction ecdsa_verif(EC, message, signature, pk), qui prend en entrée
la donnée d’une courbe elliptique EC, une chaine de caractères message, une signature signature et une
clé publique pk, et qui vérifie la validité de la signature vis à vis du message et de la clé, dans le système
de signature ECDSA.

In [50]:
#Question 11 
def verif_signature(EC,m,s,pk):
    q=EC[-1]
    b,c=s.copy() #Je copie la signature s
    G,q=EC[-2:].copy()
    p=EC[0]
    
    h = sha3_256(bytes(m, "utf-8")).hexdigest()
    h1=int(h,16)%q

    #print("c=",c, "reste =",xgcd(c,q)[0])
    premiere_partie=(h1*inverse_modulaire(c,q))%q
    deuxieme_partie=(b*inverse_modulaire(c,q))%q

    P=ec_fast_mult(EC,G,premiere_partie)
    Q=ec_fast_mult(EC,pk,deuxieme_partie)

    R=ec_add(EC,P,Q)

    x_r=R[0]%q
    return x_r==b #On met return car on a besoin d'une valeur de retour pour la question 13 (sinon ça retourne aucune signature valide)



EC2=[]

verif_signature(EC_sous_groupe,chaine_test,s,pk)


True

### Question 12.– 
Stocker les paramètres de la courbe Curve25519, puis vérifier que l’entier q est premier et
que qG = O.

In [51]:
#Question 12
a1=57896044618658097711785492504343953926634992332820282019728791901641727051837
b1=398341948620716521344
q1 = 2**252 + 27742317777372353535851937790883648493
G1=[2, 2587177637973221124604506650587198648324617568965701997790174145338249409797]
p1=(2**255-19)


EC25519=[p1,a1,b1,G1,q1]
print(isPrime(q1))


ec_fast_mult(EC25519,G1,q1)

True


'ZERO'

Pour terminer cet exercice, dans l’archive disponible à cette adresse :
https://lvzl.fr/teaching/2023-24/docs/cp/devoirs/devoir2.zip
vous trouverez différents fichiers.
1. Un fichier 4 nommé rfc8031.txt contenant une spécification de l’utilisation de la courbe
Curve25519 en cryptographie.
2. Un fichier nommé publickey.txt, qui contient deux lignes qui représentent les deux coordronnées
(x et y, dans l’ordre) d’une clé publique ECDSA instanciée avec Curve25519.
3. 100 fichiers nommés signature<ii>.txt où ii va de 00 à 99, de deux lignes chacun. Ces fichiers
représentent chacun une signature potentielle du contenu de rfc8031.txt avec la clé privée associée
à publickey.txt (vous n’avez pas cette clé privée, c’est normal). La première ligne de chaque fichier
correspond à l’entier b de la signature, la seconde ligne à c.
### Question 13.– 
Déterminer, parmi ces 100 signatures, l’unique signature valide du message consistant en
le texte contenu dans le fichier rfc8031.txt.

In [52]:
#Question 13

h_long = sha3_256(bytes(chaine_test, 'utf-8')).hexdigest()

with open('publickey.txt', 'r') as fichier_cle_publique:
    # Lire les deux lignes du fichier
    premiere_ligne =fichier_cle_publique.readline()
    deuxieme_ligne =fichier_cle_publique.readline()

pk1 = [int(premiere_ligne), int(deuxieme_ligne)]

with open("rfc8031.txt", 'r') as texte_rfc:
    texte =texte_rfc.read()

    
compteur_arret=0
for i in range(0, 100):
    if i <= 9:
        numero ="0" + str(i)
    else:
        numero =str(i)
    #print("signature"+a+".txt")

    with open("signature" + numero + ".txt", 'r') as fichier_signature:

        # Lire les deux lignes du fichier
        premiere_ligne_b =fichier_signature.readline()
        deuxieme_ligne_c =fichier_signature.readline()
        signature_fichier =[int(premiere_ligne_b), int(deuxieme_ligne_c)]
        #print("signature" + numero + ".txt")


    if verif_signature(EC25519,texte,signature_fichier,pk1) == True:
          
        print("La signature", i, "est valide")
        compteur_arret=1
        break
    fichier_signature.close()

fichier_cle_publique.close()
texte_rfc.close()
if compteur_arret == 0:
    print("Aucune signature valide trouvée.")
    

La signature 37 est valide


### Bonus

Tentative d'appliquer la méthode décrite dans le documents RFC8031

In [28]:
import math


def random_bits(bit_length):
    # Générer un nombre aléatoire avec la taille de bits donnée
    random_number = random.getrandbits(bit_length)
    
    # Convertir le nombre aléatoire en chaîne binaire et la remplir avec des zéros à gauche si nécessaire
    binary_string = bin(random_number)[2:].zfill(bit_length)
    
    return int(binary_string,2)

def transformation_bit():
    random_i=random_bits(8*32)
    random_r=random_bits(8*32)
    #random_i,random_r=bin(random_i),bin(random_r)

    liste_1111_ver1,liste_1111_ver2=1,1

    random_i_1=int(bin(random_i)[0:9],2)
    random_i_2=int(bin(random_i)[-8:],2)
    
    taille_random_i_1=math.ceil(log2(random_i_1))
    taille_random_i_2=math.ceil(log2(random_i_2))
    for i in range(0,taille_random_i_1-3-1):
        liste_1111_ver1=(liste_1111_ver1<<1)+1
    
    for i in range(0,taille_random_i_2-3-1):
        liste_1111_ver2=(liste_1111_ver2<<1)+1


    taille_random_i=math.ceil(log2(random_i))
   
    print("taille 1111",math.ceil(log2(liste_1111_ver1)))
    print("taille de random_i_1",int(math.log2(random_i_1)) + 1)


    liste_1100_ver_1=liste_1111_ver1-3
    liste_000100_ver_1=(random_i_1&liste_1100_ver_1)

    liste_1100_ver_2=liste_1111_ver2-3
    liste_000100_ver_2=(random_i_2&liste_1100_ver_2)

    

    print("Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i \n",bin(liste_1111_ver1))
    print("Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin \n",bin(liste_1100_ver_1))
    #liste_000100_ver_1_ver_1_ver_1=liste_1100_ver_1&liste_1111
    print("Le nombre qu'on veut changer random_i_1 et sa taille :\n",bin(random_i_1),)
    print("Normalement on a mis les deux bits les plus significatifs à 0\n",bin(liste_000100_ver_1))
    print("Transition \n")
 
    print("taille 1111",math.ceil(log2(liste_1111_ver2)))
    print("taille de random_i_2",int(math.log2(random_i_2)) + 1)

    print("Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i \n",bin(liste_1111_ver2))
    print("Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin \n",bin(liste_1100_ver_2))
    #liste_000100_ver_1_ver_1_ver_1=liste_1100_ver_1&liste_1111
    print("Le nombre qu'on veut changer random_i_2 :\n",bin(random_i_2))
    print("Normalement on a mis les deux bits les plus significatifs à 0\n",bin(liste_000100_ver_2))

    fixed_i = liste_000100_ver_1 + int(bin(random_i)[9:-8],2) + liste_000100_ver_2

    liste_1111_ver1,liste_1111_ver2=1,1

    random_r_1=int(bin(random_r)[0:9],2)
    random_r_2=int(bin(random_r)[-8:],2)
    taille_random_r=math.ceil(log2(random_r))


    taille_random_r_1=math.ceil(log2(random_r_1))
    taille_random_r_2=math.ceil(log2(random_r_2))

    for i in range(0,taille_random_r_1-3-1):
        liste_1111_ver1=(liste_1111_ver1<<1)+1
    
    for i in range(0,taille_random_r_2-3-1):
        liste_1111_ver2=(liste_1111_ver2<<1)+1

    liste_1100_ver_1=liste_1111_ver1-3
    liste_000100_ver_1=(random_r_1&liste_1100_ver_1)
    liste_1100_ver_2=liste_1111_ver2-3
    liste_000100_ver_2=(random_r_2&liste_1111_ver2)&liste_1100_ver_2

    fixed_j=liste_000100_ver_1 + int(bin(random_r)[9:-8],2) +liste_000100_ver_2
    


    return[fixed_i,fixed_j]


transformation_bit()




taille 1111 4
taille de random_i_1 7
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b1100
Le nombre qu'on veut changer random_i_1 et sa taille :
 0b1011111
Normalement on a mis les deux bits les plus significatifs à 0
 0b1100
Transition 

taille 1111 5
taille de random_i_2 8
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b11111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b11100
Le nombre qu'on veut changer random_i_2 :
 0b11110000
Normalement on a mis les deux bits les plus significatifs à 0
 0b10000


[1306240937421283857797517116128141935824328186622103880450556718206223358,
 2065957986437284250013802122287600446644009744054476956491215597420305141]

In [29]:
fixed_i,fixed_r=transformation_bit()

def keygen_private(fixed_i,fixed_r,EC):
    d_i = fixed_i.to_bytes(32, byteorder='little')
    d_r= fixed_r.to_bytes(32, byteorder='little')
    pub_i=ec_fast_mult(EC25519,EC25519[-2],d_i) #ou calculer ec_fast_mult(G,donnees_encodee1) avec les données de la courbe
    pub_r=ec_fast_mult(EC25519,EC25519[-2],d_r)

    Shared_secret=ec_fast_mult(EC25519,pub_i,d_i)
    if Shared_secret==ec_fast_mult(EC25519,pub_r,d_r) :  #et la aussi on doit faire ec_fast_mult(G,donnees_encodee2)
        return Shared_secret  #PROBLEME: ça ne peut pas être la clé privée
    else:
        print("Le secret n'est pas commun")


taille 1111 4
taille de random_i_1 7
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b1100
Le nombre qu'on veut changer random_i_1 et sa taille :
 0b1110000
Normalement on a mis les deux bits les plus significatifs à 0
 0b0
Transition 

taille 1111 5
taille de random_i_2 8
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b11111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b11100
Le nombre qu'on veut changer random_i_2 :
 0b10001100
Normalement on a mis les deux bits les plus significatifs à 0
 0b1100
